ArXiv Metadata Analysis

Capstone Project, DSI-911 cohort, Lisa Paul

Run this First
to read-in JSON data
and perform initial data cleaning steps





In [1]:
import pandas as pd


In [2]:
data_path = '../data/'
subdir = 'chunks_json/'

#50_000 articles, from 2007-2008
curr_file = 'arxiv_meta_aa.json'

df = pd.read_json(data_path+subdir + curr_file, lines = True)

In [3]:

#prove it worked:
df.head(3)

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,None,A fully differential calculation in perturba...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2008-11-26,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,..."
1,704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,None,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...","[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-12-13,"[[Streinu, Ileana, ], [Theran, Louis, ]]"
2,704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",None,None,None,physics.gen-ph,None,The evolution of Earth-Moon system is descri...,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2008-01-13,"[[Pan, Hongjun, ]]"


In [4]:
df.shape, df.columns

((50000, 14),
 Index(['id', 'submitter', 'authors', 'title', 'comments', 'journal-ref', 'doi',
        'report-no', 'categories', 'license', 'abstract', 'versions',
        'update_date', 'authors_parsed'],
       dtype='object'))

In [5]:
#This includes "multiple category" 
#3869, out of 50K articles in df
len(df['categories'].unique())

3869

In [6]:
#Should I data clean the categories?

#check format of dirty categories
print(df['categories'][1])

#I see that any spliting will be "on the _"

math.CO cs.CG


In [7]:
#count unique w/o whitespace
    #(~ operator suggested by chatgpt, to negate the boolean series returned by str.contains)
len(df['categories'][~df['categories'].str.contains('\s')].unique())
#120 unique categories!


#this could be sufficient, depending on ttl number using these in the 50K line file??

# List of 120 unique values without whitespace
unique_categories = df['categories'][~df['categories'].str.contains('\s')].unique()
# Count the number of rows with these unique values
count_rows = df[df['categories'].isin(unique_categories)].shape[0]

# Print the count
print(count_rows)
#there are 32031 - that's 60% of the original dataframe. 
    #yes, that's sufficient

32031


Data cleaning choice:
cull the rows with multiple categories
(I really don't want 200 OHE columns if I can help it)


In [8]:
# Drop rows where 'categories' column contains multiple values
df = df[~df['categories'].str.contains('\s')]

df.shape #confirm



(32031, 14)

In [9]:
#should I drop more columns?
#not now

In [10]:
df.head(1)

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,None,A fully differential calculation in perturba...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2008-11-26,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,..."


In [11]:
csv_partial = data_path + curr_file.rstrip('.json')
csv_file = csv_partial + '-single-cat' + '.csv'

df.to_csv(csv_file, index=False)